# Predicting Remaining Useful Life (advanced)
<p style="margin:30px">
    <img style="display:inline; margin-right:50px" width=50% src="https://www.featuretools.com/wp-content/uploads/2017/12/FeatureLabs-Logo-Tangerine-800.png" alt="Featuretools" />
    <img style="display:inline" width=15% src="https://upload.wikimedia.org/wikipedia/commons/e/e5/NASA_logo.svg" alt="NASA" />
</p>

This notebook has a more advanced workflow than [the other notebook](Simple%20Featuretools%20RUL%20Demo.ipynb) for predicting Remaining Useful Life (RUL). If you are a new to either this dataset or Featuretools, I would recommend reading the other notebook first. 

*If you're running this notebook yourself, please [download](https://ti.arc.nasa.gov/c/13/) the dataset into the `data` folder in this repository.*

## Highlights
* Demonstrate how novel entityset structures improve predictive accuracy
* Build custom primitives using time-series functions from [tsfresh](https://github.com/blue-yonder/tsfresh)
* Improve Mean Absolute Error by tuning hyper parameters with [BTB](https://github.com/HDI-Project/BTB)

Here is a collection of scores from a run of both notebooks. Because of the randomness in the Random Forest Regressor and how we choose labels from the Train data, scores are subject to change.

|                                 | Train |  Test |
|---------------------------------|---------------|
| Median Baseline                 | 62.55 | 50.55 |
| Simple Featuretools             | 41.18 | 39.56 |
| Advanced: Custom Primitives     | 39.55 | 41.02 |
| Advanced: Hyperparameter Tuning | 27.63 | 13.36 |


# Step 1: Load Data
Here we load in the train data using the same function we used in the previous notebook:

In [1]:
import numpy as np
import pandas as pd
import featuretools as ft
import utils

data_path = 'data/RUL_train.txt'
data = utils.load_data(data_path)

data.head()

,engine_no,time_in_cycles,operational_setting_1,operational_setting_2,operational_setting_3,sensor_measurement_1,sensor_measurement_2,sensor_measurement_3,sensor_measurement_4,sensor_measurement_5,...,sensor_measurement_14,sensor_measurement_15,sensor_measurement_16,sensor_measurement_17,sensor_measurement_18,sensor_measurement_19,sensor_measurement_20,sensor_measurement_21,index,time
index,,,,,,,,,,,,,,,,,,,,,
0,1,1,42.0049,0.8400,100.0,445.00,549.68,1343.43,1112.93,3.91,...,8074.83,9.3335,0.02,330,2212,100.00,10.62,6.3670,0,2000-01-01 00:00:00
1,1,2,20.0020,0.7002,100.0,491.19,606.07,1477.61,1237.50,9.35,...,8046.13,9.1913,0.02,361,2324,100.00,24.37,14.6552,1,2000-01-01 00:00:01
2,1,3,42.0038,0.8409,100.0,445.00,548.95,1343.12,1117.05,3.91,...,8066.62,9.4007,0.02,329,2212,100.00,10.48,6.4213,2,2000-01-01 00:00:02
3,1,4,42.0000,0.8400,100.0,445.00,548.70,1341.24,1118.03,3.91,...,8076.05,9.3369,0.02,328,2212,100.00,10.54,6.4176,3,2000-01-01 00:00:03
4,1,5,25.0063,0.6207,60.0,462.54,536.10,1255.23,1033.59,7.05,...,7865.80,10.8366,0.02,305,1915,84.93,14.03,8.6754,4,2000-01-01 00:00:04


We also make cutoff times by selecting a random cutoff time from the life of each engine:

In [2]:
cutoff_times = utils.make_cutoff_times(data)

cutoff_times.head()

,engine_no,cutoff_time,RUL
index,,,
1,1,2000-01-01 00:04:05,75
2,2,2000-01-01 00:05:43,276
3,3,2000-01-01 00:11:14,252
4,4,2000-01-01 00:19:27,33
5,5,2000-01-01 00:20:30,163


We're going to do something fancy for our entityset. The values for `operational_setting` 1-3 are continuous but create an implicit relation between different engines. If two engines have a similar `operational_setting`, it could indicate that we should expect the sensor measurements to mean similar things. We make clusters of those settings using `KMeans` from scikit-learn and make a new entity from the clusters.

In [3]:
from sklearn.cluster import KMeans

nclusters = 50

def make_entityset(data, nclusters, kmeans=None):
    X = data[['operational_setting_1', 'operational_setting_2', 'operational_setting_3']]
    if kmeans:
        kmeans=kmeans
    else:
        kmeans = KMeans(n_clusters=nclusters).fit(X)
    data['settings_clusters'] = kmeans.predict(X)
    
    es = ft.EntitySet('Dataset')
    es.entity_from_dataframe(dataframe=data,
                             entity_id='recordings',
                             index='index',
                             time_index='time')

    es.normalize_entity(base_entity_id='recordings', 
                        new_entity_id='engines',
                        index='engine_no')
    
    es.normalize_entity(base_entity_id='recordings', 
                        new_entity_id='cycles',
                        index='time_in_cycles')
    
    es.normalize_entity(base_entity_id='recordings', 
                        new_entity_id='settings_clusters',
                        index='settings_clusters')
    
    return es, kmeans
es, kmeans = make_entityset(data, nclusters)
es

Entityset: Dataset
  Entities:
    recordings (shape = [61249, 29])
    cycles (shape = [543, 2])
    settings_clusters (shape = [50, 2])
    engines (shape = [249, 2])
  Relationships:
    recordings.engine_no -> engines.engine_no
    recordings.time_in_cycles -> cycles.time_in_cycles
    recordings.settings_clusters -> settings_clusters.settings_clusters

# Step 2: DFS and Creating a Model
In addition to changing our `EntitySet` structure, we're also going to use some time series primitives from the package [tsfresh](https://github.com/blue-yonder/tsfresh).

In [4]:
from tsfresh.feature_extraction.feature_calculators import number_peaks, mean_abs_change
from featuretools.primitives import make_agg_primitive
import featuretools.variable_types as vtypes

def tsf_numpeaks(series):
    return number_peaks(series, 3)

MeanAbsChange = make_agg_primitive(mean_abs_change,
                                   input_types=[vtypes.Numeric],
                                   return_type=vtypes.Numeric,
                                   name="mean_abs_change")

NumPeaks = make_agg_primitive(lambda x: number_peaks(x, 3),
                              input_types=[vtypes.Numeric],
                              return_type=vtypes.Numeric,
                              name="number_peaks")


from featuretools.primitives import Sum, Mean, Std, Skew, Max, Min, Last, CumSum, Diff, Trend, Count
fm, features = ft.dfs(entityset=es, 
                      target_entity='engines',
                      agg_primitives=[Max, Min, Last, NumPeaks, MeanAbsChange],
                      trans_primitives=[],
                      cutoff_time=cutoff_times,
                      max_depth=3,
                      verbose=True)

/Users/featurelabs07/homeenv/lib/python2.7/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


Built 1326 features
Elapsed: 22:24 | Remaining: 00:00 | Progress: 100%|██████████|| Calculated: 249/249 cutoff times


# Step 3: Feature Selection and Scoring
Here, we'll use [Recursive Feature Elimination](http://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.RFE.html). In order to set ourselves up for later optimization, we're going to write a generic `pipeline` function which takes in a set of hyperparameters and returns a score. Our pipeline will first run `RFE` and then split the remaining data for scoring by a `RandomForestRegressor`. We're going to pass in a list of hyperparameters, which we will tune later. 

In [5]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.feature_selection import RFE

def pipeline(X, y, hyperparams):
    """ Hyperparams: [
            0: number of estimators for the random forest in RFE
            1: number of features to select
            2: number of estimators for  random forest in scoring
            3: max feats for random forest in scoring
        ]
    """
    reg = RandomForestRegressor(n_estimators=int(hyperparams[0]), n_jobs=3)
    X_train, X_test, y_train, y_test = train_test_split(X, y)
    selector = RFE(reg, int(hyperparams[1]), step=50)
    selector = selector.fit(X_train, y_train)
    max_feats = min(hyperparams[3], hyperparams[1])
    reg = RandomForestRegressor(n_estimators=int(hyperparams[2]), 
                                max_features=int(max_feats))
    reg.fit(selector.transform(X_train), y_train)
    
    preds = reg.predict(selector.transform(X_test))
    score = mean_absolute_error(preds, y_test)
    return score, (selector, reg)

X = fm.copy().fillna(0)
y = X.pop('RUL')

rfe_nest = 10
nfeats = 30
sco_nest = 10
sco_maxfeats = 20

hyperparams = [rfe_nest, nfeats, sco_nest, sco_maxfeats]
score, (selector, model) = pipeline(X, y, hyperparams)

print('Mean Abs Error: {:.2f}'.format(score))
high_imp_feats = utils.feature_importances(X.iloc[:, selector.support_], model, feats=10)

Mean Abs Error: 39.55
1: NUMBER_PEAKS(recordings.settings_clusters.NUMBER_PEAKS(recordings.sensor_measurement_8)) [0.193]
2: MAX(recordings.cycles.LAST(recordings.sensor_measurement_17)) [0.129]
3: MAX(recordings.sensor_measurement_13) [0.081]
4: MAX(recordings.cycles.LAST(recordings.sensor_measurement_11)) [0.078]
5: NUMBER_PEAKS(recordings.settings_clusters.MIN(recordings.sensor_measurement_13)) [0.078]
6: MAX(recordings.settings_clusters.LAST(recordings.sensor_measurement_3)) [0.057]
7: MEAN_ABS_CHANGE(recordings.cycles.MIN(recordings.sensor_measurement_7)) [0.037]
8: MAX(recordings.cycles.LAST(recordings.sensor_measurement_15)) [0.036]
9: MAX(recordings.settings_clusters.LAST(recordings.sensor_measurement_13)) [0.035]
10: MAX(recordings.settings_clusters.LAST(recordings.sensor_measurement_4)) [0.031]
-----



Lastly, we can use that selector and regressor to score the test values.

In [6]:
data2 = utils.load_data('data/RUL_test.txt')

es2, _ = make_entityset(data2, nclusters, kmeans=kmeans)
fm2 = ft.calculate_feature_matrix(entityset=es2, features=features, verbose=True)
X = fm2.copy().fillna(0)
y = pd.read_csv('data/RUL_test_truth.txt', sep=' ', header=-1, names=['RUL'], index_col=False)
preds2 = model.predict(selector.transform(X))
print('Mean Abs Error: {:.2f}'.format(mean_absolute_error(preds2, y)))

Elapsed: 00:59 | Remaining: 00:00 | Progress: 100%|██████████|| Calculated: 1/1 cutoff times
Mean Abs Error: 41.02


# Step 4: Hyperparameter Tuning
Because of the way we set up our pipeline, we can use a Gaussian Process to tune the hyperparameters. We will use [BTB](https://github.com/HDI-Project/BTB) from the [HDI Project](https://github.com/HDI-Project).

In [7]:
from btb.hyper_parameter import HyperParameter
from btb.tuning.gp import GP
from tqdm import tqdm

def run_btb(X, y, n=30):
    hyperparam_ranges = [
            ('selector_n_estimators', HyperParameter('int', [100, 1000])),
            ('select_n_features', HyperParameter('int', [5, 50])),
            ('model_n_estimators', HyperParameter('int', [100, 500])),
            ('model_max_feats', HyperParameter('int', [2, 20])),
    ]
    tuner = GP(hyperparam_ranges)

    tested_parameters = np.zeros((n, len(hyperparam_ranges)), dtype=object)
    scores = []
    best = 100
    
    print('[sel_n_est, sel_n_feats, model_n_est, model_max_feats]')
    for i in tqdm(range(n)):
        tuner.fit(tested_parameters[:i, :], scores)
        hyperparams = tuner.propose()

        bound = -pipeline(X, y, hyperparams)[0]
        tested_parameters[i, :] = hyperparams
        scores.append(bound)
        if -bound < best:
            best = -bound
            print('{}: New best score of {:.2f}'.format(hyperparams, -bound))
    return tested_parameters, scores

X = fm.copy().fillna(0)
y = X.pop('RUL')

tested_parameters, scores = run_btb(X, y, n=30)

  0%|          | 0/30 [00:00<?, ?it/s]

[sel_n_est, sel_n_feats, model_n_est, model_max_feats]


  3%|▎         | 1/30 [01:30<43:38, 90.29s/it]

[192.  46. 212.  20.]: New best score of 31.94


 37%|███▋      | 11/30 [47:24<1:21:53, 258.60s/it]

[968.  14. 119.   5.]: New best score of 29.35


100%|██████████| 30/30 [1:57:30<00:00, 235.02s/it]  


In [13]:
hyperparams = [192.,  46., 212.,  20.]
score, (selector, model) = pipeline(X, y, hyperparams)

print('Mean Abs Error on Train: {:.2f}'.format(score))
high_imp_feats = utils.feature_importances(X.iloc[:, selector.support_], model, feats=10)

/Users/featurelabs07/homeenv/lib/python2.7/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/featurelabs07/homeenv/lib/python2.7/site-packages/ipykernel_launcher.py:21: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


Mean Abs Error on Train: 27.63
1: MAX(recordings.sensor_measurement_11) [0.183]
2: MAX(recordings.sensor_measurement_13) [0.156]
3: MAX(recordings.sensor_measurement_4) [0.112]
4: MAX(recordings.sensor_measurement_8) [0.093]
5: MAX(recordings.sensor_measurement_3) [0.041]
6: MAX(recordings.sensor_measurement_9) [0.031]
7: MAX(recordings.sensor_measurement_2) [0.025]
8: MEAN_ABS_CHANGE(recordings.cycles.NUMBER_PEAKS(recordings.operational_setting_1)) [0.024]
9: MAX(recordings.sensor_measurement_14) [0.023]
10: LAST(recordings.cycles.MAX(recordings.sensor_measurement_11)) [0.017]
-----



In [14]:
X = fm2.copy().fillna(0)
y = pd.read_csv('data/RUL_test_truth.txt', sep=' ', header=-1, names=['RUL'], index_col=False)

preds2 = model.predict(selector.transform(X))
score2 = mean_absolute_error(preds2, y)
print('Mean Abs Error on Test: {:.2f}'.format(score2))


Mean Abs Error on Test: 13.36
